In [60]:
import sklearn,lightgbm,hyperopt
print(sklearn.__version__)
print(lightgbm.__version__)
print(hyperopt.__version__)

1.2.2
3.1.1
0.2.5


In [61]:
import pandas as pd
import numpy as np
from datetime import datetime 
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno

pd.set_option('display.max_columns', 60)
pd.set_option('display.max_rows', 400)

In [62]:
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error
import bisect
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from lightgbm import LGBMRegressor

# 구글 드라이브에서 데이터를 다운로드하는 클래스 
from DataLoader import DatasetDownloader

In [63]:
pd.set_option('display.max_columns', 30)
pd.set_option('display.max_rows', 300)

In [64]:
ID = '1M-uP5sGsIe7BOx5Oj9vxjDUxoSiHctgN' # 구글드라이브 url 토큰

download = DatasetDownloader(ID)
df_train, df_test, submit = download.dataload()

Loding data from Local...
sample_submission.csv: 3.1MB
test.csv            : 32.5MB
train.csv           : 61.4MB


In [65]:
print(df_train.shape)
df_train[:5]

(391939, 23)


,SAMPLE_ID,ARI_CO,ARI_PO,SHIP_TYPE_CATEGORY,DIST,ATA,ID,BREADTH,BUILT,DEADWEIGHT,DEPTH,DRAUGHT,GT,LENGTH,SHIPMANAGER,FLAG,U_WIND,V_WIND,AIR_TEMPERATURE,BN,ATA_LT,PORT_SIZE,CI_HOUR
0,TRAIN_000000,SG,GIW5,Container,30.881018,2018-12-17 21:29,Z618338,30.0,24,24300,10.0,10.0,16700,180.0,CQSB78,Panama,NaN,NaN,NaN,NaN,5,0.002615,3.450000
1,TRAIN_000001,IN,UJM2,Bulk,0.000000,2014-09-23 6:59,X886125,30.0,13,35900,10.0,10.0,23500,180.0,SPNO34,Marshall Islands,NaN,NaN,NaN,NaN,12,0.000217,0.000000
2,TRAIN_000002,CN,EUC8,Container,0.000000,2015-02-03 22:00,T674582,50.0,12,146000,30.0,20.0,140000,370.0,FNPK22,Malta,NaN,NaN,NaN,NaN,6,0.001614,0.000000
3,TRAIN_000003,JP,ZAG4,Container,0.000000,2020-01-17 4:02,Y847238,20.0,18,6910,10.0,10.0,5400,120.0,PBZV77,Bahamas,-3.18,-1.61,6.7,2.629350,13,0.000356,0.000000
4,TRAIN_000004,SG,GIW5,Container,27.037650,2020-01-26 7:51,A872328,50.0,10,116000,20.0,10.0,96600,300.0,GUCE76,Liberia,-0.33,-3.28,25.6,2.495953,15,0.002615,253.554444


In [66]:
print(df_test.shape)
df_test[:5]

(220491, 22)


,SAMPLE_ID,ARI_CO,ARI_PO,SHIP_TYPE_CATEGORY,DIST,ATA,ID,BREADTH,BUILT,DEADWEIGHT,DEPTH,DRAUGHT,GT,LENGTH,SHIPMANAGER,FLAG,U_WIND,V_WIND,AIR_TEMPERATURE,BN,ATA_LT,PORT_SIZE
0,TEST_000000,SG,GIW5,Container,1.826589,2020-06-18 11:58,K322654,50.0,18,117000,30.0,20.0,109000,340.0,KQMD74,Panama,0.37,1.63,27.1,1.587063,19,0.002615
1,TEST_000001,CN,WEY7,Cargo,25.399386,2021-05-26 22:20,E376681,10.0,13,3810,10.0,10.0,2560,80.0,LBYS27,Belize,-2.79,-2.33,14.2,2.663972,6,0.001028
2,TEST_000002,CN,NGG6,Container,111.079467,2019-12-16 0:09,F811111,20.0,26,10900,10.0,10.0,8440,140.0,EKFV15,Singapore,0.04,-4.91,9.3,3.255315,8,0.001743
3,TEST_000003,CA,FFM2,Bulk,9.175258,2015-11-16 5:30,A737561,30.0,9,55800,20.0,10.0,31500,190.0,MASW32,Panama,NaN,NaN,NaN,NaN,2,0.000182
4,TEST_000004,JP,QYY1,Container,0.000000,2018-10-24 1:11,A827175,30.0,19,39800,20.0,10.0,33000,220.0,SIEL54,Liberia,NaN,NaN,NaN,NaN,10,0.000552


In [67]:
train = df_train.copy()
test = df_test.copy()

In [68]:
wind_cond = (train['BN']==0) & (train['U_WIND']==0) & (train['V_WIND']==0)

train['BN'] = np.where(wind_cond, np.nan, train['BN'])
train['U_WIND'] = np.where(wind_cond, np.nan, train['U_WIND'])
train['V_WIND'] = np.where(wind_cond, np.nan, train['V_WIND'])

wind_cond = (test['BN']==0) & (test['U_WIND']==0) & (test['V_WIND']==0)

test['BN'] = np.where(wind_cond, np.nan, test['BN'])
test['U_WIND'] = np.where(wind_cond, np.nan, test['U_WIND'])
test['V_WIND'] = np.where(wind_cond, np.nan, test['V_WIND'])



In [69]:
def engineering(df) :
    #  ATA컬럼 datetime형으로 변경 후 연, 월, 시, 일 컬럼 생성
    df['ATA'] = pd.to_datetime(df['ATA'])
    df['ATA_YEAR'] = df['ATA'].dt.year
    df['ATA_MONTH'] = df['ATA'].dt.month
    df['ATA_HOUR'] = df['ATA'].dt.hour
    df['ATA_DAY'] = df['ATA'].dt.day
    
    # 항구국적 + 항구명으로 도착지 컬럼 생성
    df['ARI'] = df['ARI_CO'] + '-' + df['ARI_PO']
    # 데이터를 정렬
    names = np.sort(df['ARI'].unique())
    # 각 고유한 값에 대한 인덱스를 저장하는 딕셔너리 생성
    encoding_dict = {name: index + 1 for index, name in enumerate(names)}
    # "ARI" 열을 이용하여 새로운 변수 "ARI_ENCODING"을 생성
    df['ARI_NUM'] = df['ARI'].map(encoding_dict)
    
    # 연, 달, 일, 항구국적별 데이터 건수
    ymd_co = df.groupby(['ATA_YEAR', 'ATA_MONTH', 'ATA_DAY', 'ARI_CO'])['SAMPLE_ID'].agg('count').reset_index()
    ymd_co.rename(columns={'SAMPLE_ID':'YMD_CO_CNT'}, inplace=True)
    df = df.merge(ymd_co, on=['ATA_YEAR', 'ATA_MONTH', 'ATA_DAY', 'ARI_CO'], how='left')
    
    # 연, 달, 일, 도착지별 데이터 건수
    ymd_ari = df.groupby(['ATA_YEAR', 'ATA_MONTH', 'ATA_DAY', 'ARI'])['SAMPLE_ID'].agg('count').reset_index()
    ymd_ari.rename(columns={'SAMPLE_ID':'YMD_ARI_CNT'}, inplace=True)
    df = df.merge(ymd_ari, on=['ATA_YEAR', 'ATA_MONTH', 'ATA_DAY', 'ARI'], how='left')
    
    # 연도별 데이터 건수
    y_cnt = df.groupby(['ATA_YEAR'])['SAMPLE_ID'].agg('count').reset_index()
    y_cnt.rename(columns={'SAMPLE_ID':'Y_CNT'}, inplace=True)
    df = df.merge(y_cnt, on='ATA_YEAR', how='left')
    
    # DIST, PORT_SIZE 비율
    df['DIST_PORT_SIZE_RATIO'] = df['DIST'] / df['PORT_SIZE']
    df['DIST_PORT_SIZE_RATIO'] = np.where(df['DIST_PORT_SIZE_RATIO']==np.inf, np.nan, df['DIST_PORT_SIZE_RATIO'])
    
    # 선박 종류가 컨테이너인지 아닌지
    df['IS_CONTAINER'] = np.where(df['SHIP_TYPE_CATEGORY']=='Container', 1, 0)
    
    # 풍속, 풍향
    df['WIND_SPEED'] = np.sqrt(df['U_WIND']**2 + df['V_WIND']**2)
    df['WIND_DIRECTION'] = np.arctan2(df['U_WIND'], df['V_WIND'])
    
    # 항구국적별 DIST 평균, 표준편차
    co_dist = df.groupby(['ARI_CO'])['DIST'].agg(['mean', 'std']).reset_index()
    co_dist.columns = ['ARI_CO', 'CO_DIST_MEAN', 'CO_DIST_STD']
    df = df.merge(co_dist, on=['ARI_CO'], how='left')
    
    # 도착지별 DIST 평균, 표준편차
    ari_dist = df.groupby(['ARI'])['DIST'].agg(['mean', 'std']).reset_index()
    ari_dist.columns = ['ARI', 'ARI_DIST_MEAN', 'ARI_DIST_STD']
    df = df.merge(ari_dist, on=['ARI'], how='left')
    
    # 선박 종류별 DIST 평균, 표준편차
    type_dist = df.groupby(['SHIP_TYPE_CATEGORY'])['DIST'].agg(['mean', 'std']).reset_index()
    type_dist.columns = ['SHIP_TYPE_CATEGORY', 'TYPE_DIST_MEAN', 'TYPE_DIST_STD']
    df = df.merge(type_dist, on='SHIP_TYPE_CATEGORY', how='left')
    
    # DRAUGHT 별 DIST 평균, 표준편차
    draught_dist = df.groupby(['DRAUGHT'])['DIST'].agg(['mean', 'std']).reset_index()
    draught_dist.columns = ['DRAUGHT', 'DRAUGHT_DIST_MEAN', 'DRAUGHT_DIST_STD']
    df = df.merge(draught_dist, on='DRAUGHT', how='left')
    
    # 선박의 국적별 DIST 평균, 표준편차
    flag_dist = df.groupby(['FLAG'])['DIST'].agg(['mean', 'std']).reset_index()
    flag_dist.columns = ['FLAG', 'FLAG_DIST_MEAN', 'FLAG_DIST_STD']
    df = df.merge(flag_dist, on=['FLAG'], how='left')
    
    # 온도, DIST 비율
    df['DIST_TEMP_RATIO'] = df['DIST'] / df['AIR_TEMPERATURE']
    df['DIST_TEMP_RATIO'] = np.where(df['DIST_TEMP_RATIO']== np.inf, np.nan, df['DIST_TEMP_RATIO'])
    
    # BN, DIST 비율
    df['DIST_BN_RATIO'] = df['DIST'] / df['BN']
    df['DIST_BN_RATIO'] = np.where(df['DIST_BN_RATIO']== np.inf, np.nan, df['DIST_BN_RATIO'])
    
    # 항구국적별 port_size 평균, 표준편차
    co_port_size = df.groupby(['ARI_CO'])['PORT_SIZE'].agg(['mean', 'std']).reset_index()
    co_port_size.columns = ['ARI_CO', 'CO_PORT_SIZE_MEAN', 'CO_PORT_SIZE_STD']
    df = df.merge(co_port_size, on='ARI_CO', how='left')
    
    return df

In [70]:
train = engineering(train)
test = engineering(test)

train.shape, test.shape

((391939, 50), (220491, 49))

### 피쳐 엔지니어링
- Engineering_02 + 
- 1) 도착항구별 대기시간 평균 표준편차
- 2) 도착항구국적별 대기시간 평균 표준편차
- 3) 선박 종류별 대기시간 평균 표준편차
- 4) ID별 대기시간 평균 표준편차
- 5) FLAG별 대기시간 평균 표준편차
- 6) ATA_MONTH별 대기시간 평균 표준편차
- 7) ATA_YEAR별 대기시간 평균 표준편차
- 8) ATA_DAY별 대기시간 평균 표준편차

In [71]:
def engineering_2(train, test) : 
    # 항구국적별 대기시간 평균, 표준편차
    co_target = train.groupby('ARI_CO')['CI_HOUR'].agg(['mean', 'std']).reset_index()
    co_target.columns = ['ARI_CO', 'CO_TARGET_MEAN', 'CO_TARGET_STD']
    train = train.merge(co_target, on='ARI_CO', how='left')
    test = test.merge(co_target, on='ARI_CO', how='left')
    
    # 도착지별 대기시간 평균, 표준편차
    ari_target = train.groupby('ARI')['CI_HOUR'].agg(['mean', 'std']).reset_index()
    ari_target.columns = ['ARI', 'ARI_TARGET_MEAN', 'ARI_TARGET_STD']
    train = train.merge(ari_target, on='ARI', how='left')
    test = test.merge(ari_target, on='ARI', how='left')
    
    # 선박 종류별 대기시간 평균 표준편차
    type_target = train.groupby('SHIP_TYPE_CATEGORY')['CI_HOUR'].agg(['mean', 'std']).reset_index()
    type_target.columns = ['SHIP_TYPE_CATEGORY', 'TYPE_TARGET_MEAN', 'TYPE_TARGET_STD']
    train = train.merge(type_target, on='SHIP_TYPE_CATEGORY', how='left')
    test = test.merge(type_target, on='SHIP_TYPE_CATEGORY', how='left')
    
    # 선박ID별 대기시간 평균 표준편차
    id_target = train.groupby('ID')['CI_HOUR'].agg(['mean', 'std']).reset_index()
    id_target.columns = ['ID', 'ID_TARGET_MEAN', 'ID_TARGET_STD']
    train = train.merge(id_target, on='ID', how='left')
    test = test.merge(id_target, on='ID', how='left')
    
    # flag별 대기시간 평균 표준편차
    flag_target = train.groupby('FLAG')['CI_HOUR'].agg(['mean', 'std']).reset_index()
    flag_target.columns = ['FLAG', 'FLAG_TARGET_MEAN', 'FLAG_TARGET_STD']
    train = train.merge(flag_target, on='FLAG', how='left')
    test = test.merge(flag_target, on='FLAG', how='left')
    
    # 달별 대기시간 평균 표준편차
    month_target = train.groupby('ATA_MONTH')['CI_HOUR'].agg(['mean', 'std']).reset_index()
    month_target.columns = ['ATA_MONTH', 'MONTH_TARGET_MEAN', 'MONTH_TARGET_STD']
    train = train.merge(month_target, on='ATA_MONTH', how='left')
    test = test.merge(month_target, on='ATA_MONTH', how='left')
    
    # 연별 대기시간 평균 표준편차
    year_target = train.groupby('ATA_YEAR')['CI_HOUR'].agg(['mean', 'std']).reset_index()
    year_target.columns = ['ATA_YEAR', 'YEAR_TARGET_MEAN', 'YEAR_TARGET_STD']
    train = train.merge(year_target, on='ATA_YEAR', how='left')
    test = test.merge(year_target, on='ATA_YEAR', how='left')
    
    # 일별 대기시간 평균 표준편차
    day_target = train.groupby('ATA_DAY')['CI_HOUR'].agg(['mean', 'std']).reset_index()
    day_target.columns = ['ATA_DAY', 'DAY_TARGET_MEAN', 'DAY_TARGET_STD']
    train = train.merge(day_target, on='ATA_DAY', how='left')
    test = test.merge(day_target, on='ATA_DAY', how='left')
    
    return train, test

In [72]:
train, test = engineering_2(train, test)
train.shape, test.shape

((391939, 66), (220491, 65))

In [73]:
train.isnull().sum()[train.isnull().sum() > 0]

BREADTH                   1
DEPTH                     1
DRAUGHT                   1
LENGTH                    1
U_WIND               176711
V_WIND               176711
AIR_TEMPERATURE      164630
BN                   176711
WIND_SPEED           176711
WIND_DIRECTION       176711
DRAUGHT_DIST_MEAN         1
DRAUGHT_DIST_STD          1
FLAG_DIST_STD             9
DIST_TEMP_RATIO      165791
DIST_BN_RATIO        176711
ID_TARGET_STD          3803
FLAG_TARGET_STD           9
dtype: int64

In [74]:
test.isnull().sum()[test.isnull().sum() > 0]

U_WIND              98984
V_WIND              98984
AIR_TEMPERATURE     92246
BN                  98984
WIND_SPEED          98984
WIND_DIRECTION      98984
FLAG_DIST_STD          10
DIST_TEMP_RATIO     92900
DIST_BN_RATIO       98984
ID_TARGET_MEAN       2207
ID_TARGET_STD        4700
FLAG_TARGET_MEAN        1
FLAG_TARGET_STD        18
dtype: int64

### 결측치 처리 및 제거
ARI = TW-JWI3 없애줘도 될 것 같다는 의견입니다. 

In [75]:
import pandas as pd
from tqdm import tqdm

interpolate_sample = train.copy()

def interpolate_df(df, method='linear'):
    new_df = pd.DataFrame()
    
    # ARI 별로 데이터를 분할하여 보간
    for ari_value in tqdm(df['ARI'].unique()):
        ari_df = df[df['ARI'] == ari_value].sort_values(by='ATA').copy()
        
        # 결측치 여부 확인
        missing_values = ari_df.isnull().any()
        
        # 결측치가 있는 열만 선택
        columns_with_missing_values = missing_values[missing_values].index
        
        if not columns_with_missing_values.empty:
            print(f"ARI {ari_value}에 대한 결측치 열: {', '.join(columns_with_missing_values)}")
        
        # ATA를 인덱스로 설정
        ari_df.set_index('ATA', inplace=True)
        
        # 시간대별로 각 컬럼을 보간
        for column in ['AIR_TEMPERATURE', 'BN', 'U_WIND', 'V_WIND']:
            ari_df[column] = ari_df[column].interpolate(method=method, limit_direction='both')
        
        # 결측치를 이전 값으로 채우기
        ari_df = ari_df.fillna(method='ffill')
        
        # 다른 컬럼에 결측치가 있을 경우 해당 데이터프레임의 평균값으로 채우기
        for other_column in columns_with_missing_values:
            if other_column not in ['AIR_TEMPERATURE', 'BN', 'U_WIND', 'V_WIND']:
                ari_df[other_column] = ari_df[other_column].fillna(ari_df[other_column].mean())
        
        # 보간된 데이터를 new_df에 추가
        new_df = pd.concat([new_df, ari_df.reset_index()])
    
    return new_df.sort_values(by='SAMPLE_ID')

# interpolate_df 함수를 호출하여 데이터 보간
interpolated_data = interpolate_df(interpolate_sample)
interpolated_data 

  1%|▊                                                                                 | 1/104 [00:00<00:20,  4.94it/s]

ARI SG-GIW5에 대한 결측치 열: U_WIND, V_WIND, AIR_TEMPERATURE, BN, WIND_SPEED, WIND_DIRECTION, DIST_TEMP_RATIO, DIST_BN_RATIO, ID_TARGET_STD
ARI IN-UJM2에 대한 결측치 열: U_WIND, V_WIND, AIR_TEMPERATURE, BN, WIND_SPEED, WIND_DIRECTION, DIST_TEMP_RATIO, DIST_BN_RATIO, ID_TARGET_STD


  3%|██▎                                                                               | 3/104 [00:00<00:13,  7.49it/s]

ARI CN-EUC8에 대한 결측치 열: U_WIND, V_WIND, AIR_TEMPERATURE, BN, WIND_SPEED, WIND_DIRECTION, DIST_TEMP_RATIO, DIST_BN_RATIO, ID_TARGET_STD
ARI JP-ZAG4에 대한 결측치 열: U_WIND, V_WIND, AIR_TEMPERATURE, BN, WIND_SPEED, WIND_DIRECTION, DIST_TEMP_RATIO, DIST_BN_RATIO, ID_TARGET_STD


  5%|███▉                                                                              | 5/104 [00:00<00:11,  8.79it/s]

...

ARI ID-DIN2에 대한 결측치 열: ID_TARGET_STD
ARI VN-ONW1에 대한 결측치 열: U_WIND, V_WIND, AIR_TEMPERATURE, BN, WIND_SPEED, WIND_DIRECTION, DIST_TEMP_RATIO, DIST_BN_RATIO, ID_TARGET_STD


 99%|███████████████████████████████████████████████████████████████████████████████▏| 103/104 [00:15<00:00,  6.64it/s]

ARI RU-ZME5에 대한 결측치 열: U_WIND, V_WIND, BN, WIND_SPEED, WIND_DIRECTION, DIST_TEMP_RATIO, DIST_BN_RATIO, ID_TARGET_STD
ARI CN-RGT8에 대한 결측치 열: U_WIND, V_WIND, AIR_TEMPERATURE, BN, WIND_SPEED, WIND_DIRECTION, DIST_TEMP_RATIO, DIST_BN_RATIO


100%|████████████████████████████████████████████████████████████████████████████████| 104/104 [00:15<00:00,  6.67it/s]


,ATA,SAMPLE_ID,ARI_CO,ARI_PO,SHIP_TYPE_CATEGORY,DIST,ID,BREADTH,BUILT,DEADWEIGHT,DEPTH,DRAUGHT,GT,LENGTH,SHIPMANAGER,...,CO_TARGET_STD,ARI_TARGET_MEAN,ARI_TARGET_STD,TYPE_TARGET_MEAN,TYPE_TARGET_STD,ID_TARGET_MEAN,ID_TARGET_STD,FLAG_TARGET_MEAN,FLAG_TARGET_STD,MONTH_TARGET_MEAN,MONTH_TARGET_STD,YEAR_TARGET_MEAN,YEAR_TARGET_STD,DAY_TARGET_MEAN,DAY_TARGET_STD
17500,2018-12-17 21:29:00,TRAIN_000000,SG,GIW5,Container,30.881018,Z618338,30.0,24,24300,10.0,10.0,16700,180.0,CQSB78,...,289.632484,109.031581,289.632484,39.811365,165.410920,68.555680,147.136057,64.873071,170.208030,64.751276,178.464769,60.376507,163.056411,63.843852,175.336872
12,2014-09-23 06:59:00,TRAIN_000001,IN,UJM2,Bulk,0.000000,X886125,30.0,13,35900,10.0,10.0,23500,180.0,SPNO34,...,145.474300,64.678644,154.686604,79.357251,159.363268,28.722667,37.359211,71.262264,167.836434,62.256246,172.431481,95.657226,241.614838,60.382820,169.045538
256,2015-02-03 22:00:00,TRAIN_000002,CN,EUC8,Container,0.000000,T674582,50.0,12,146000,30.0,20.0,140000,370.0,FNPK22,...,126.702344,55.686871,173.882240,39.811365,165.410920,5.909969,9.441246,64.500805,193.800460,59.689659,164.832945,56.825993,168.889246,60.698039,161.813712
1839,2020-01-17 04:02:00,TRAIN_000003,JP,ZAG4,Container,0.000000,Y847238,20.0,18,6910,10.0,10.0,5400,120.0,PBZV77,...,146.204704,45.561169,151.285651,39.811365,165.410920,10.206132,20.948055,54.749401,147.068087,62.618716,167.404020,63.534533,173.926701,63.843852,175.336872
22638,2020-01-26 07:51:00,TRAIN_000004,SG,GIW5,Container,27.037650,A872328,50.0,10,116000,20.0,10.0,96600,300.0,GUCE76,...,289.632484,109.031581,289.632484,39.811365,165.410920,37.072858,149.581410,68.482039,179.345656,62.618716,167.404020,63.534533,173.926701,61.857355,170.962646
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3751,2017-06-06 05:02:00,TRAIN_391934,JP,QYY1,Container,0.000000,Y375615,20.0,27,6820,10.0,10.0,4820,110.0,KEJZ24,...,146.204704,19.513979,94.608875,39.811365,165.410920,20.338156,77.990531,61.773755,146.760917,59.902860,162.682641,54.868054,156.360945,61.462201,168.496479
21383,2019-10-16 00:36:00,TRAIN_391935,SG,GIW5,Bulk,5.884603,K635567,10.0,12,3160,10.0,10.0,1600,80.0,JLTM64,...,289.632484,109.031581,289.632484,79.357251,159.363268,719.303359,829.961152,77.495206,225.397103,62.837179,172.643108,58.126869,167.055771,59.801774,169.729427
872,2021-03-23 22:35:00,TRAIN_391936,US,QGN3,Bulk,70.660241,J284147,30.0,8,60300,20.0,10.0,34800,200.0,YERJ68,...,166.046124,54.982687,135.679768,79.357251,159.363268,68.230556,52.033691,57.271856,167.610951,61.172841,170.858605,67.562373,175.861856,60.382820,169.045538
96,2015-01-08 07:15:00,TRAIN_391937,TW,JWI3,Container,9.448179,J644215,30.0,29,23800,10.0,10.0,16900,170.0,HCZK58,...,100.893512,23.272262,105.084912,39.811365,165.410920,18.629395,86.798264,41.074514,165.189030,62.618716,167.404020,56.825993,168.889246,59.613858,166.397394


In [76]:
interpolated_data 

,ATA,SAMPLE_ID,ARI_CO,ARI_PO,SHIP_TYPE_CATEGORY,DIST,ID,BREADTH,BUILT,DEADWEIGHT,DEPTH,DRAUGHT,GT,LENGTH,SHIPMANAGER,...,CO_TARGET_STD,ARI_TARGET_MEAN,ARI_TARGET_STD,TYPE_TARGET_MEAN,TYPE_TARGET_STD,ID_TARGET_MEAN,ID_TARGET_STD,FLAG_TARGET_MEAN,FLAG_TARGET_STD,MONTH_TARGET_MEAN,MONTH_TARGET_STD,YEAR_TARGET_MEAN,YEAR_TARGET_STD,DAY_TARGET_MEAN,DAY_TARGET_STD
17500,2018-12-17 21:29:00,TRAIN_000000,SG,GIW5,Container,30.881018,Z618338,30.0,24,24300,10.0,10.0,16700,180.0,CQSB78,...,289.632484,109.031581,289.632484,39.811365,165.410920,68.555680,147.136057,64.873071,170.208030,64.751276,178.464769,60.376507,163.056411,63.843852,175.336872
12,2014-09-23 06:59:00,TRAIN_000001,IN,UJM2,Bulk,0.000000,X886125,30.0,13,35900,10.0,10.0,23500,180.0,SPNO34,...,145.474300,64.678644,154.686604,79.357251,159.363268,28.722667,37.359211,71.262264,167.836434,62.256246,172.431481,95.657226,241.614838,60.382820,169.045538
256,2015-02-03 22:00:00,TRAIN_000002,CN,EUC8,Container,0.000000,T674582,50.0,12,146000,30.0,20.0,140000,370.0,FNPK22,...,126.702344,55.686871,173.882240,39.811365,165.410920,5.909969,9.441246,64.500805,193.800460,59.689659,164.832945,56.825993,168.889246,60.698039,161.813712
1839,2020-01-17 04:02:00,TRAIN_000003,JP,ZAG4,Container,0.000000,Y847238,20.0,18,6910,10.0,10.0,5400,120.0,PBZV77,...,146.204704,45.561169,151.285651,39.811365,165.410920,10.206132,20.948055,54.749401,147.068087,62.618716,167.404020,63.534533,173.926701,63.843852,175.336872
22638,2020-01-26 07:51:00,TRAIN_000004,SG,GIW5,Container,27.037650,A872328,50.0,10,116000,20.0,10.0,96600,300.0,GUCE76,...,289.632484,109.031581,289.632484,39.811365,165.410920,37.072858,149.581410,68.482039,179.345656,62.618716,167.404020,63.534533,173.926701,61.857355,170.962646
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3751,2017-06-06 05:02:00,TRAIN_391934,JP,QYY1,Container,0.000000,Y375615,20.0,27,6820,10.0,10.0,4820,110.0,KEJZ24,...,146.204704,19.513979,94.608875,39.811365,165.410920,20.338156,77.990531,61.773755,146.760917,59.902860,162.682641,54.868054,156.360945,61.462201,168.496479
21383,2019-10-16 00:36:00,TRAIN_391935,SG,GIW5,Bulk,5.884603,K635567,10.0,12,3160,10.0,10.0,1600,80.0,JLTM64,...,289.632484,109.031581,289.632484,79.357251,159.363268,719.303359,829.961152,77.495206,225.397103,62.837179,172.643108,58.126869,167.055771,59.801774,169.729427
872,2021-03-23 22:35:00,TRAIN_391936,US,QGN3,Bulk,70.660241,J284147,30.0,8,60300,20.0,10.0,34800,200.0,YERJ68,...,166.046124,54.982687,135.679768,79.357251,159.363268,68.230556,52.033691,57.271856,167.610951,61.172841,170.858605,67.562373,175.861856,60.382820,169.045538
96,2015-01-08 07:15:00,TRAIN_391937,TW,JWI3,Container,9.448179,J644215,30.0,29,23800,10.0,10.0,16900,170.0,HCZK58,...,100.893512,23.272262,105.084912,39.811365,165.410920,18.629395,86.798264,41.074514,165.189030,62.618716,167.404020,56.825993,168.889246,59.613858,166.397394


-> 'WIND_SPEED', 'WIND_DIRECTION'은 시간대에 따라 달라질 것 같아서 따로 처리해주면 좋을듯

In [77]:
def interpolate_test_data(train, test, method='both'):
    # Train 데이터에서 ARI와 ATA 별 평균값 계산
    numeric_columns = train.select_dtypes(include=['float', 'int']).columns
    mean_values = train.groupby(['ARI', 'ATA'])[numeric_columns].mean().reset_index()

    # Test 데이터 복사
    interpolate_test = test.copy()

    for ari_value in tqdm(interpolate_test['ARI'].unique()):
        # Train 데이터에서 해당 ARI 값에 대한 평균값 가져오기
        mean_value = mean_values[(mean_values['ARI'] == ari_value)].sort_values(by='ATA').copy()

        if not mean_value.empty:
            test_subset = interpolate_test[interpolate_test['ARI'] == ari_value].sort_values(by='ATA').copy()

            # 결측치가 있는 열만 선택
            missing_columns = test_subset.columns[test_subset.isnull().any()]

            for column in missing_columns:
                # 결측치를 Train 평균값으로 채우기
                test_subset[column].fillna(mean_value[column].values[0], inplace=True)# 다른 컬럼에 결측치가 있을 경우 해당 데이터프레임의 평균값으로 채우기
                for other_column in  missing_columns:
                    if other_column not in ['AIR_TEMPERATURE', 'BN', 'U_WIND', 'V_WIND']:
                        test_subset[column] = test_subset[column].fillna(mean_value[other_column].mean())
        

            # 보간된 데이터를 Test 데이터에 적용
            interpolate_test.loc[interpolate_test['ARI'] == ari_value] = test_subset
        else:
            # 처리할 평균값이 없는 경우에 대한 처리
            print(f"No mean values found for ARI {ari_value}")

    # 결과 데이터프레임을 SAMPLE ID로 정렬
    interpolate_test.sort_values(by='SAMPLE_ID', inplace=True)

    return interpolate_test
interpolate_test = interpolate_test_data(interpolated_data, test, method='both')

100%|████████████████████████████████████████████████████████████████████████████████| 104/104 [00:10<00:00,  9.46it/s]


In [78]:
interpolated_data.isnull().sum()[train.isnull().sum() > 0]

BREADTH              0
DEPTH                0
DRAUGHT              0
LENGTH               0
U_WIND               6
V_WIND               6
AIR_TEMPERATURE      6
BN                   6
WIND_SPEED           6
WIND_DIRECTION       6
DRAUGHT_DIST_MEAN    0
DRAUGHT_DIST_STD     0
FLAG_DIST_STD        0
DIST_TEMP_RATIO      6
DIST_BN_RATIO        6
ID_TARGET_STD        0
FLAG_TARGET_STD      0
dtype: int64

In [79]:
interpolate_test .isnull().sum()[test.isnull().sum() > 0]

U_WIND              5
V_WIND              5
AIR_TEMPERATURE     5
BN                  5
WIND_SPEED          5
WIND_DIRECTION      5
FLAG_DIST_STD       0
DIST_TEMP_RATIO     5
DIST_BN_RATIO       5
ID_TARGET_MEAN      0
ID_TARGET_STD       0
FLAG_TARGET_MEAN    0
FLAG_TARGET_STD     0
dtype: int64

### BUILT , BN 계급 범주화

In [80]:
def preprocessing_cat(x) :
    x['BUILT_CAT'] = np.NaN
    x.loc[x['BUILT'] < 4, 'BUILT_CAT'] = 0 # 정상데이터가 있는지도 몰루
    x.loc[((4 <= x['BUILT']) & (x['BUILT'] < 6)), 'BUILT_CAT'] = 1
    x.loc[((6 <= x['BUILT']) & (x['BUILT'] < 12)), 'BUILT_CAT'] = 2
    x.loc[((12 <= x['BUILT']) & (x['BUILT'] < 15)), 'BUILT_CAT'] = 3
    x.loc[((15 <= x['BUILT']) & (x['BUILT'] < 18)), 'BUILT_CAT'] = 4
    x.loc[((18 <= x['BUILT']) & (x['BUILT'] < 25)), 'BUILT_CAT']= 5
    x.loc[((25 <= x['BUILT']) & (x['BUILT'] < 30)), 'BUILT_CAT'] = 6
    x.loc[((30 <= x['BUILT']) & (x['BUILT'] < 40)), 'BUILT_CAT'] = 7
    x.loc[((40 <= x['BUILT']) & (x['BUILT'] < 50)), 'BUILT_CAT'] = 8
    x.loc[((50 <= x['BUILT']) & (x['BUILT'] < 60)), 'BUILT_CAT'] = 9
    x.loc[((60 <= x['BUILT']) & (x['BUILT'] < 65)), 'BUILT_CAT'] = 10
    x.loc[((65 <= x['BUILT']) & (x['BUILT'] < 75)), 'BUILT_CAT'] = 11
    x.loc[((75 <= x['BUILT']) & (x['BUILT'] < 85)), 'BUILT_CAT'] = 12
    x.loc[85 <= x['BUILT'], 'BUILT_CAT'] = 13
    
    
    x['BN_CAT'] = np.NaN
    x.loc[x['BN'] < 1, 'BN_CAT'] = 0
    x.loc[(1 <= x['BN']) & (x['BN'] < 2), 'BN_CAT'] = 1
    x.loc[(2 <= x['BN']) & (x['BN'] < 3), 'BN_CAT'] = 2
    x.loc[(3 <= x['BN']) & (x['BN'] < 4), 'BN_CAT'] = 3
    x.loc[(4 <= x['BN']) & (x['BN'] < 5), 'BN_CAT'] = 4
    x.loc[(5 <= x['BN']) & (x['BN'] < 6), 'BN_CAT'] = 5
    x.loc[(6 <= x['BN']) & (x['BN'] < 7), 'BN_CAT'] = 6
    x.loc[(7 <= x['BN']) & (x['BN'] < 8), 'BN_CAT'] = 7
    x.loc[(8 <= x['BN']) & (x['BN'] < 9), 'BN_CAT'] = 8
    x.loc[(9 <= x['BN']) & (x['BN'] < 10), 'BN_CAT'] = 9
    x.loc[(10 <= x['BN']) & (x['BN'] < 11), 'BN_CAT'] = 10
    x.loc[(11 <= x['BN']) & (x['BN'] < 12), 'BN_CAT'] = 11
    x.loc[(12 <= x['BN']) & (x['BN'] < 13), 'BN_CAT'] = 12
    x.loc[(13 <= x['BN']), 'BN_CAT'] = 13

    return x

#### 레이블인코딩

In [81]:
# 카테고리 컬럼은 모두 label Encoding 수행
cat_columns = train.dtypes[train.dtypes == object].index.tolist()
cat_columns

['SAMPLE_ID',
 'ARI_CO',
 'ARI_PO',
 'SHIP_TYPE_CATEGORY',
 'ID',
 'SHIPMANAGER',
 'FLAG',
 'ARI']

In [82]:
interpolated_data.shape, interpolate_test.shape

((391939, 66), (220491, 65))

In [83]:
cat_columns.remove('SAMPLE_ID')
encoders = {}

interpolated_data = preprocessing_cat(interpolated_data)
interpolate_test = preprocessing_cat(interpolate_test)

def label_encoding(df_train, df_test):
    for col in cat_columns : 
        le = LabelEncoder()
        df_train[col] = le.fit_transform(df_train[col].astype(str))
        le_classes_set = set(le.classes_)
        
        df_test[col] = df_test[col].map(lambda x : '-1' if x not in le_classes_set else x)
        le_classes = le.classes_.tolist()
        bisect.insort_left(le_classes, '-1')
        le.classes_ = np.array(le_classes)
        df_test[col] = le.transform(df_test[col].astype(str))
        encoders[col] = le
    return df_train, df_test  


interpolated_data,interpolate_test = label_encoding(interpolated_data,interpolate_test)

#### 스케일링
- https://mkjjo.github.io/python/2019/01/10/scaler.html

In [84]:
train, test = interpolated_data.copy(), interpolate_test.copy()

# 수치형 피쳐 중 DIST, DEADWEIGHT, GT Robust ,'AIR_TEMPERATURE','PORT_SIZE'스케일링
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler

def scaling(train, test) : 
    cols = ['DIST', 'DEADWEIGHT', 'GT','AIR_TEMPERATURE','PORT_SIZE']
    for col in cols : 
        rb = RobustScaler()
        train[col] = rb.fit_transform(train[col].values.reshape(-1,1))
        test[col] = rb.transform(test[col].values.reshape(-1,1))
    
    return train, test

train, test = scaling(train, test)

In [85]:
train.columns

Index(['ATA', 'SAMPLE_ID', 'ARI_CO', 'ARI_PO', 'SHIP_TYPE_CATEGORY', 'DIST',
       'ID', 'BREADTH', 'BUILT', 'DEADWEIGHT', 'DEPTH', 'DRAUGHT', 'GT',
       'LENGTH', 'SHIPMANAGER', 'FLAG', 'U_WIND', 'V_WIND', 'AIR_TEMPERATURE',
       'BN', 'ATA_LT', 'PORT_SIZE', 'CI_HOUR', 'ATA_YEAR', 'ATA_MONTH',
       'ATA_HOUR', 'ATA_DAY', 'ARI', 'ARI_NUM', 'YMD_CO_CNT', 'YMD_ARI_CNT',
       'Y_CNT', 'DIST_PORT_SIZE_RATIO', 'IS_CONTAINER', 'WIND_SPEED',
       'WIND_DIRECTION', 'CO_DIST_MEAN', 'CO_DIST_STD', 'ARI_DIST_MEAN',
       'ARI_DIST_STD', 'TYPE_DIST_MEAN', 'TYPE_DIST_STD', 'DRAUGHT_DIST_MEAN',
       'DRAUGHT_DIST_STD', 'FLAG_DIST_MEAN', 'FLAG_DIST_STD',
       'DIST_TEMP_RATIO', 'DIST_BN_RATIO', 'CO_PORT_SIZE_MEAN',
       'CO_PORT_SIZE_STD', 'CO_TARGET_MEAN', 'CO_TARGET_STD',
       'ARI_TARGET_MEAN', 'ARI_TARGET_STD', 'TYPE_TARGET_MEAN',
       'TYPE_TARGET_STD', 'ID_TARGET_MEAN', 'ID_TARGET_STD',
       'FLAG_TARGET_MEAN', 'FLAG_TARGET_STD', 'MONTH_TARGET_MEAN',
       'MONTH_TARGET

In [86]:
ignore_feature = ['ARI_CO','ARI_PO','BN','BUILT','ATA','WIND_DIRECTION','WIND_SPEED','ATA_LT', 'AIR_TEMPERATURE',
                 'DRAUGHT_DIST_MEAN','DRAUGHT_DIST_STD','DEPTH', 'DRAUGHT',
                 'TYPE_DIST_MEAN', 'TYPE_DIST_STD','TYPE_TARGET_MEAN', 'TYPE_TARGET_STD','BREADTH' ] 

In [87]:
train.shape, test.shape

((391939, 68), (220491, 67))

In [88]:
# 수치형 피쳐 중 DIST, DEADWEIGHT, GT Robust ,'AIR_TEMPERATURE','PORT_SIZE'스케일링
from sklearn.preprocessing import RobustScaler

cols = ['DIST', 'DEADWEIGHT', 'GT','AIR_TEMPERATURE','PORT_SIZE']

for col in cols : 
    rb = RobustScaler()
    train[col] = rb.fit_transform(train[col].values.reshape(-1,1))
    test[col] = rb.transform(test[col].values.reshape(-1,1))

In [89]:
train['CI_HOUR'] = np.log1p(train['CI_HOUR'])

https://medium.com/analytics-vidhya/hyperparameter-tuning-hyperopt-bayesian-optimization-for-xgboost-and-neural-network-8aedf278a1c9

In [109]:
import xgboost as xgb
from sklearn.metrics import mean_absolute_error
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

SEED = 2023
VALID_SIZE = 0.2
TARGET = 'CI_HOUR'

#-------------------------------------------------#
# Utility functions

def intersect(l_1, l_2):
    return list(set(l_1) & set(l_2))


def get_features(train, test):
    intersecting_features = intersect(train.columns, test.columns)
    intersecting_features.remove('SAMPLE_ID')
    intersecting_features.remove('ATA')
    return sorted(intersecting_features)

#-------------------------------------------------#
# Scoring and optimization functions

space = {
    'n_estimators': 500,
    'eta': hp.quniform('eta', 0.025, 0.5, 0.025),
    'max_depth': hp.choice('max_depth', [int(x) for x in np.arange(1, 14, dtype=int)]),
    'min_child_weight': hp.quniform('min_child_weight', 1, 6, 1),
    'subsample': hp.quniform('subsample', 0.5, 1, 0.05),
    'gamma': hp.quniform('gamma', 0.5, 1, 0.05),
    'colsample_bytree': hp.quniform('colsample_bytree', 0.5, 1, 0.05),
    'reg_alpha': hp.uniform('reg_alpha', 0, 1),  # Add 'reg_alpha' here
    'eval_metric': 'mae',
    'objective': 'reg:linear',
    'nthread': 4,
    'booster': 'gbtree',
    'tree_method': 'exact',
    'silent': 1,
    'seed': SEED
}

def score(params):
    print("Training with params: ")
    print(params)
    num_round = int(params['n_estimators'])
    del params['n_estimators']
    
    model = xgb.XGBRegressor(
    n_estimators=space['n_estimators'],
    max_depth=space['max_depth'],
    gamma=space['gamma'],
    reg_alpha=space['reg_alpha'],  # 수정된 부동 소수점 값 사용
    min_child_weight=space['min_child_weight'],
    colsample_bytree=space['colsample_bytree']
)
    
    x_train = xgb.DMatrix(x_train_features, label=y_train)
    x_valid = xgb.DMatrix(x_valid_features, label=y_valid)
    
    watchlist = [(x_valid, 'eval'), (x_train, 'train')]
    model = xgb.train(params, x_train, num_round,
                          evals=watchlist,early_stopping_rounds=50,
                          verbose_eval=True)
    
    pred = model.predict(x_valid)
    
    mae=  mean_absolute_error(y_valid, pred)
    print ("SCORE:", mae)
    return {'loss':  (-1) * mae, 'status': STATUS_OK}


def optimize(random_state=SEED):
    trials = Trials()
    # Use the fmin function from Hyperopt to find the best hyperparameters
    best = fmin(score, space, algo=tpe.suggest,
                max_evals=100,trials=trials)
    return best

In [110]:
FEATURES = get_features(train, test)
print(FEATURES)

['AIR_TEMPERATURE', 'ARI', 'ARI_CO', 'ARI_DIST_MEAN', 'ARI_DIST_STD', 'ARI_NUM', 'ARI_PO', 'ARI_TARGET_MEAN', 'ARI_TARGET_STD', 'ATA_DAY', 'ATA_HOUR', 'ATA_LT', 'ATA_MONTH', 'ATA_YEAR', 'BN', 'BN_CAT', 'BREADTH', 'BUILT', 'BUILT_CAT', 'CI_HOUR', 'CO_DIST_MEAN', 'CO_DIST_STD', 'CO_PORT_SIZE_MEAN', 'CO_PORT_SIZE_STD', 'CO_TARGET_MEAN', 'CO_TARGET_STD', 'DAY_TARGET_MEAN', 'DAY_TARGET_STD', 'DEADWEIGHT', 'DEPTH', 'DIST', 'DIST_BN_RATIO', 'DIST_PORT_SIZE_RATIO', 'DIST_TEMP_RATIO', 'DRAUGHT', 'DRAUGHT_DIST_MEAN', 'DRAUGHT_DIST_STD', 'FLAG', 'FLAG_DIST_MEAN', 'FLAG_DIST_STD', 'FLAG_TARGET_MEAN', 'FLAG_TARGET_STD', 'GT', 'ID', 'ID_TARGET_MEAN', 'ID_TARGET_STD', 'IS_CONTAINER', 'LENGTH', 'MONTH_TARGET_MEAN', 'MONTH_TARGET_STD', 'PORT_SIZE', 'SHIPMANAGER', 'SHIP_TYPE_CATEGORY', 'TYPE_DIST_MEAN', 'TYPE_DIST_STD', 'TYPE_TARGET_MEAN', 'TYPE_TARGET_STD', 'U_WIND', 'V_WIND', 'WIND_DIRECTION', 'WIND_SPEED', 'YEAR_TARGET_MEAN', 'YEAR_TARGET_STD', 'YMD_ARI_CNT', 'YMD_CO_CNT', 'Y_CNT']


In [111]:
# Extract the train and valid (used for validation) dataframes from the train_df

x_train, x_valid = train_test_split(train, test_size=VALID_SIZE,random_state=SEED)
x_train_features = x_train[FEATURES]
x_valid_features = x_valid[FEATURES]
y_train = x_train[TARGET]
y_valid = x_valid[TARGET]

print('The training set is of length: ', len(x_train.index))
print('The validation set is of length: ', len(x_valid.index))

The training set is of length:  313551
The validation set is of length:  78388


In [ ]:
best_hyperparams = optimize()
print("The best hyperparameters are: ", "\n")
print(best_hyperparams)

Training with params:                                                                                                  
{'booster': 'gbtree', 'colsample_bytree': 1.0, 'eta': 0.025, 'eval_metric': 'mae', 'gamma': 0.75, 'max_depth': 4, 'min_child_weight': 4.0, 'n_estimators': 500, 'nthread': 4, 'objective': 'reg:linear', 'reg_alpha': 0.8987399640569644, 'seed': 2023, 'silent': 1, 'subsample': 0.9, 'tree_method': 'exact'}
[0]	eval-mae:1.83430	train-mae:1.83745                                                                                 
[1]	eval-mae:1.78849	train-mae:1.79155                                                                                 
[2]	eval-mae:1.74379	train-mae:1.74679                                                                                 
[3]	eval-mae:1.70021	train-mae:1.70314                                                                                 
[4]	eval-mae:1.65772	train-mae:1.66057                                                           

[64]	eval-mae:0.36331	train-mae:0.36395                                                                                
[65]	eval-mae:0.35423	train-mae:0.35486                                                                                
[66]	eval-mae:0.34539	train-mae:0.34600                                                                                
[67]	eval-mae:0.33677	train-mae:0.33737                                                                                
[68]	eval-mae:0.32835	train-mae:0.32894                                                                                
[69]	eval-mae:0.32015	train-mae:0.32072                                                                                
[70]	eval-mae:0.31216	train-mae:0.31271                                                                                
[71]	eval-mae:0.30436	train-mae:0.30491                                                                                
[72]	eval-mae:0.29677	train-mae:0.29730 

[132]	eval-mae:0.06517	train-mae:0.06529                                                                               
[133]	eval-mae:0.06355	train-mae:0.06367                                                                               
[134]	eval-mae:0.06197	train-mae:0.06208                                                                               
[135]	eval-mae:0.06043	train-mae:0.06054                                                                               
[136]	eval-mae:0.05892	train-mae:0.05903                                                                               
[137]	eval-mae:0.05746	train-mae:0.05756                                                                               
[138]	eval-mae:0.05603	train-mae:0.05613                                                                               
[139]	eval-mae:0.05464	train-mae:0.05474                                                                               
[140]	eval-mae:0.05328	train-mae:0.05338

[200]	eval-mae:0.01190	train-mae:0.01192                                                                               
[201]	eval-mae:0.01161	train-mae:0.01163                                                                               
[202]	eval-mae:0.01133	train-mae:0.01135                                                                               
[203]	eval-mae:0.01105	train-mae:0.01108                                                                               
[204]	eval-mae:0.01079	train-mae:0.01081                                                                               
[205]	eval-mae:0.01052	train-mae:0.01055                                                                               
[206]	eval-mae:0.01027	train-mae:0.01029                                                                               
[207]	eval-mae:0.01002	train-mae:0.01004                                                                               
[208]	eval-mae:0.00978	train-mae:0.00980

[268]	eval-mae:0.00281	train-mae:0.00281                                                                               
[269]	eval-mae:0.00277	train-mae:0.00277                                                                               
[270]	eval-mae:0.00273	train-mae:0.00274                                                                               
[271]	eval-mae:0.00269	train-mae:0.00270                                                                               
[272]	eval-mae:0.00266	train-mae:0.00267                                                                               
[273]	eval-mae:0.00262	train-mae:0.00263                                                                               
[274]	eval-mae:0.00259	train-mae:0.00259                                                                               
[275]	eval-mae:0.00257	train-mae:0.00257                                                                               
[276]	eval-mae:0.00253	train-mae:0.00253

...

[101]	eval-mae:0.03715	train-mae:0.03680                                                                               
[102]	eval-mae:0.03708	train-mae:0.03673                                                                               
[103]	eval-mae:0.03686	train-mae:0.03650                                                                               
[104]	eval-mae:0.03634	train-mae:0.03600                                                                               
[105]	eval-mae:0.03620	train-mae:0.03586                                                                               
[106]	eval-mae:0.03599	train-mae:0.03565                                                                               
[107]	eval-mae:0.03595	train-mae:0.03561                                                                               
[108]	eval-mae:0.03575	train-mae:0.03541                                                                               
[109]	eval-mae:0.03560	train-mae:0.03526

[169]	eval-mae:0.02682	train-mae:0.02650                                                                               
[170]	eval-mae:0.02674	train-mae:0.02642                                                                               
[171]	eval-mae:0.02670	train-mae:0.02638                                                                               
[172]	eval-mae:0.02661	train-mae:0.02629                                                                               
[173]	eval-mae:0.02666	train-mae:0.02634                                                                               
[174]	eval-mae:0.02654	train-mae:0.02621                                                                               
[175]	eval-mae:0.02651	train-mae:0.02619                                                                               
[176]	eval-mae:0.02649	train-mae:0.02616                                                                               
[177]	eval-mae:0.02648	train-mae:0.02615

[237]	eval-mae:0.02407	train-mae:0.02380                                                                               
[238]	eval-mae:0.02399	train-mae:0.02373                                                                               
[239]	eval-mae:0.02399	train-mae:0.02373                                                                               
[240]	eval-mae:0.02399	train-mae:0.02373                                                                               
[241]	eval-mae:0.02398	train-mae:0.02371                                                                               
[242]	eval-mae:0.02398	train-mae:0.02371                                                                               
[243]	eval-mae:0.02398	train-mae:0.02371                                                                               
[244]	eval-mae:0.02398	train-mae:0.02371                                                                               
[245]	eval-mae:0.02397	train-mae:0.02371

[305]	eval-mae:0.02327	train-mae:0.02301                                                                               
[306]	eval-mae:0.02327	train-mae:0.02301                                                                               
[307]	eval-mae:0.02327	train-mae:0.02301                                                                               
[308]	eval-mae:0.02327	train-mae:0.02301                                                                               
[309]	eval-mae:0.02327	train-mae:0.02301                                                                               
[310]	eval-mae:0.02327	train-mae:0.02301                                                                               
[311]	eval-mae:0.02327	train-mae:0.02301                                                                               
[312]	eval-mae:0.02327	train-mae:0.02301                                                                               
[313]	eval-mae:0.02327	train-mae:0.02301

[20]	eval-mae:0.07207	train-mae:0.07080                                                                                
[21]	eval-mae:0.06269	train-mae:0.06140                                                                                
[22]	eval-mae:0.05482	train-mae:0.05352                                                                                
[23]	eval-mae:0.04826	train-mae:0.04696                                                                                
[24]	eval-mae:0.04276	train-mae:0.04148                                                                                
[25]	eval-mae:0.03826	train-mae:0.03698                                                                                
[26]	eval-mae:0.03561	train-mae:0.03429                                                                                
[27]	eval-mae:0.03222	train-mae:0.03092                                                                                
[28]	eval-mae:0.02945	train-mae:0.02816 

In [ ]:
# best = {'colsample_bytree': 0.8500000000000001, 'eta': 0.025, 'gamma': 0.9, 'max_depth': 0, 'min_child_weight': 6.0, 'n_estimators': 750.0, 'reg_alpha': 0.30103334043995533, 'subsample': 0.8}

In [ ]:
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

X = train[FEATURES]
# X = train[select_features]
y = train[TARGET]
x_train, x_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=2023)

# 모델 초기화 및 학습
best_hyperparams['n_estimators'] = int(best_hyperparams['n_estimators'])
model = XGBRegressor(**best_hyperparams)
model.fit(x_train, y_train, eval_metric='mae', verbose=True)

# 모델 평가
y_pred = model.predict(x_valid)
mae = mean_absolute_error(y_valid, y_pred)
print(f"Mean Squared Error on Validation Data: {mae}")

In [ ]:
pred_v = model.predict(test[FEATURES])

# Target값 복원 
pred_v = np.expm1(pred_v)
pred_v = np.where(pred_v <= 0, 0, pred_v)
# print('MAE :', mean_absolute_error(y_valid, pred_v))

In [ ]:
import os 
test['CI_HOUR'] = pred_v

# version관리 
version = 0
while os.path.isfile('tuning_xgb_hyopt_v'+str(version)+'.csv'):
      version+=1
test[['SAMPLE_ID', 'CI_HOUR']].to_csv('tuning_xgb_hyopt_v'+str(version)+'.csv',index=False)